#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [1]:
import pandas as pd
import pymongo
import numpy as np
import json

In [2]:
client = pymongo.MongoClient("mongo:27017")
database = client['exercices']
collection = database['kickstarter']

In [3]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [4]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

In [5]:
df_ks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                150000 non-null  int64  
 1   name              149998 non-null  object 
 2   category          150000 non-null  object 
 3   main_category     150000 non-null  object 
 4   currency          150000 non-null  object 
 5   deadline          150000 non-null  object 
 6   goal              150000 non-null  object 
 7   launched          150000 non-null  object 
 8   pledged           150000 non-null  object 
 9   state             150000 non-null  object 
 10  backers           150000 non-null  object 
 11  country           150000 non-null  object 
 12  usd pledged       148518 non-null  object 
 13  usd_pledged_real  150000 non-null  float64
dtypes: float64(1), int64(1), object(12)
memory usage: 16.0+ MB


## Question 0

### Nettoyer les données

In [6]:
# Comme vu précédemment, nous avons des colonnes qui nous posent problèmes,
# nous allons donc supprimer dans ces colonnes les valeurs qui posent problème,
# puis nous allons convertir leurs valeurs en les types qui correspondent

In [7]:
df_ks = df_ks[df_ks['goal'].astype(str).str.contains('.', na=False, regex=False)]

In [8]:
df_ks['goal']= df_ks['goal'].astype(float)

In [9]:
df_ks = df_ks[df_ks['pledged'].astype(str).str.contains('.', na=False, regex=False)]

In [10]:
df_ks['pledged']= df_ks['pledged'].astype(float)

In [11]:
df_ks['backers']= df_ks['backers'].astype(int)

In [12]:
df_ks = df_ks[df_ks['usd pledged'].astype(str).str.contains('.', na=False, regex=False)]

In [13]:
df_ks['usd pledged']= df_ks['usd pledged'].astype(float)

In [14]:
df_ks.info()
df_ks

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148517 entries, 0 to 149999
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                148517 non-null  int64  
 1   name              148515 non-null  object 
 2   category          148517 non-null  object 
 3   main_category     148517 non-null  object 
 4   currency          148517 non-null  object 
 5   deadline          148517 non-null  object 
 6   goal              148517 non-null  float64
 7   launched          148517 non-null  object 
 8   pledged           148517 non-null  float64
 9   state             148517 non-null  object 
 10  backers           148517 non-null  int64  
 11  country           148517 non-null  object 
 12  usd pledged       148517 non-null  float64
 13  usd_pledged_real  148517 non-null  float64
dtypes: float64(4), int64(2), object(8)
memory usage: 17.0+ MB


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000.0,2011-08-17 06:31:31,1145.00,canceled,24,US,1145.000000,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000.0,2012-07-23 20:46:48,1851.00,failed,28,US,1851.000000,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000.0,2016-07-01 21:55:54,7534.00,successful,254,US,3796.000000,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000.0,2017-08-28 15:47:02,16298.00,successful,367,US,2670.000000,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000.0,2016-01-25 17:37:10,1.00,failed,1,CA,0.708148,0.738225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,574841011,Scutes Headphones - A new way to listen to you...,Product Design,Design,USD,2014-11-14,40000.0,2014-10-14 20:44:37,3664.00,canceled,43,US,3664.000000,3664.000000
149996,235190237,My Drunk @$$ Reviews,Webseries,Film & Video,USD,2012-05-18,1000.0,2012-04-18 04:42:47,0.00,failed,0,US,0.000000,0.000000
149997,758540498,Freedom Planet - High Speed Platform Game,Video Games,Games,USD,2013-02-15,2000.0,2013-01-16 00:53:32,25472.69,successful,1105,US,25472.690000,25472.690000
149998,1624952469,Clown vs Mime,Webseries,Film & Video,USD,2011-05-27,1000.0,2011-04-27 01:22:52,76.00,failed,3,US,76.000000,76.000000


In [15]:
# Etant donné qu'il y a beaucoup de données
# nous pouvont récupérer qu'une partie des données,
# comme par exemple les projets ayant réussi ou sont en cours

In [16]:
df_ks['state'].unique()

array(['canceled', 'failed', 'successful', 'live', 'suspended'],
      dtype=object)

In [17]:
# df_ks= df_ks[df_ks['state'].isin(['successful', 'live'])]
# df_ks

### Importer les données

In [18]:
# On choisit les identifiants
df_ks= df_ks.rename(columns={"ID":"_id"})

In [19]:
payload = json.loads(df_ks.to_json(orient='records'))
collection.delete_many({}) #pour nettoyer la collection
collection.insert_many(payload)

In [20]:
collection.find_one()

{'_id': 872782264,
 'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
 'category': 'Rock',
 'main_category': 'Music',
 'currency': 'USD',
 'deadline': '2011-09-16',
 'goal': 2000.0,
 'launched': '2011-08-17 06:31:31',
 'pledged': 1145.0,
 'state': 'canceled',
 'backers': 24,
 'country': 'US',
 'usd pledged': 1145.0,
 'usd_pledged_real': 1145.0}

## Question 1  

In [21]:
# 5 projets ayant reçu le plus de promesse de dons
cur= collection.find().sort([("pledged",-1)]).limit(5)
list(cur)

[{'_id': 342886736,
  'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler",
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2014-08-30',
  'goal': 50000.0,
  'launched': '2014-07-08 10:14:37',
  'pledged': 13285226.36,
  'state': 'successful',
  'backers': 62642,
  'country': 'US',
  'usd pledged': 13285226.36,
  'usd_pledged_real': 13285226.36},
 {'_id': 2103598555,
  'name': 'Pebble 2, Time 2 + All-New Pebble Core',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2016-06-30',
  'goal': 1000000.0,
  'launched': '2016-05-24 15:49:52',
  'pledged': 12779843.49,
  'state': 'successful',
  'backers': 66673,
  'country': 'US',
  'usd pledged': 12779843.49,
  'usd_pledged_real': 12779843.49},
 {'_id': 2111201788,
  'name': 'Expect the Unexpected. digiFilmï¿½ Camera by YASHICA',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'HKD',
  'deadline': '2017-

## Question 2

In [22]:
# nombre de projets ayant atteint leur but
nb_successful = collection.find({"state":"successful"}).count()
nb_successful

/tmp/ipykernel_12/3143706418.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  nb_successful = collection.find({"state":"successful"}).count()


52998

## Question 3

In [23]:
# nombre de projets pour chaque catégorie
cur= collection.aggregate([
    {"$group" : {
        "_id" : "$main_category",
        "number_projects" : {"$sum" : 1}
    }}
])
list(cur)

[{'_id': 'Dance', 'number_projects': 1488},
 {'_id': 'Food', 'number_projects': 9805},
 {'_id': 'Theater', 'number_projects': 4305},
 {'_id': 'Art', 'number_projects': 11260},
 {'_id': 'Crafts', 'number_projects': 3463},
 {'_id': 'Music', 'number_projects': 19657},
 {'_id': 'Film & Video', 'number_projects': 24983},
 {'_id': 'Design', 'number_projects': 11911},
 {'_id': 'Fashion', 'number_projects': 8943},
 {'_id': 'Photography', 'number_projects': 4176},
 {'_id': 'Journalism', 'number_projects': 1807},
 {'_id': 'Publishing', 'number_projects': 15721},
 {'_id': 'Games', 'number_projects': 13988},
 {'_id': 'Technology', 'number_projects': 12871},
 {'_id': 'Comics', 'number_projects': 4139}]

## Question 4

In [24]:
# nombre de projets français ayant été instanciés avant 2016
nb_fr_before_2016 = collection.find({
    "$and":[
        {"country" : "FR"},
        {"launched" : {"$lte" : "2016-01-01 00:00:00"}}
    ]
})
nb_fr_before_2016 = nb_fr_before_2016.count()
nb_fr_before_2016

/tmp/ipykernel_12/197053112.py:8: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  nb_fr_before_2016 = nb_fr_before_2016.count()


330

## Question 5

In [25]:
# projets américains ayant demandé plus de 200 000 dollars
cur = collection.find({
    "$and":[
        {"country" : "US"},
        {"goal" : {"$gte" : 200000}}
    ]
})
list(cur)

[{'_id': 655043686,
  'name': 'Far from Par is a movie about a man and a talking golf ball.',
  'category': 'Comedy',
  'main_category': 'Film & Video',
  'currency': 'USD',
  'deadline': '2014-12-05',
  'goal': 200000.0,
  'launched': '2014-10-06 21:20:06',
  'pledged': 10.0,
  'state': 'failed',
  'backers': 2,
  'country': 'US',
  'usd pledged': 10.0,
  'usd_pledged_real': 10.0},
 {'_id': 866634482,
  'name': 'A CALL TO ADVENTURE',
  'category': 'Film & Video',
  'main_category': 'Film & Video',
  'currency': 'USD',
  'deadline': '2012-09-14',
  'goal': 287000.0,
  'launched': '2012-08-13 23:14:02',
  'pledged': 1465.0,
  'state': 'failed',
  'backers': 11,
  'country': 'US',
  'usd pledged': 1465.0,
  'usd_pledged_real': 1465.0},
 {'_id': 993194166,
  'name': 'Storybricks, the storytelling online RPG',
  'category': 'Video Games',
  'main_category': 'Games',
  'currency': 'USD',
  'deadline': '2012-06-01',
  'goal': 250000.0,
  'launched': '2012-05-01 20:49:58',
  'pledged': 23680.

## Question 6 

In [26]:
# nombre de projet ayant "Sport" dans leur nom
#méthode 1
nb_Sport_projects = collection.find({"name" : {"$regex" : "Sport"}}).count()
nb_Sport_projects

/tmp/ipykernel_12/2446262478.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  nb_Sport_projects = collection.find({"name" : {"$regex" : "Sport"}}).count()


321

In [27]:
# méthode 2
nb_Sport_projects = collection.count_documents({"name" : {"$regex" : "Sport"}})
nb_Sport_projects

321